## Advanced Explainability Analysis for Credit Risk Assessment


In [1]:
import os
import json
import pickle
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

import shap
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

from tensorflow import keras

ROOT = os.path.abspath(os.getcwd())
PROJECT_ROOT = os.path.abspath(os.path.join(ROOT, '..'))

MODELS_DIR = os.path.join(PROJECT_ROOT, 'models')
DATASET_DIR = os.path.join(PROJECT_ROOT, 'dataset')
ARTIFACTS_DIR = os.path.join(PROJECT_ROOT, 'artifacts')
OUTPUT_DIR = os.path.join(ARTIFACTS_DIR, '04b_images')

os.makedirs(OUTPUT_DIR, exist_ok=True)

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")


## Data and Model Loading


In [2]:
X_train = pd.read_pickle(os.path.join(DATASET_DIR, 'X_train.pkl'))
X_test = pd.read_pickle(os.path.join(DATASET_DIR, 'X_test.pkl'))
y_test = pd.read_pickle(os.path.join(DATASET_DIR, 'y_test.pkl'))

if isinstance(y_test, pd.DataFrame):
    y_test = y_test.iloc[:, 0]

feature_names = X_train.columns.tolist()

print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")
print(f"Number of features: {len(feature_names)}")


Training data shape: (26064, 17)
Test data shape: (6517, 17)
Number of features: 17


In [3]:
def _require_model_file(filename):
    path = os.path.join(MODELS_DIR, filename)
    if not os.path.exists(path):
        raise FileNotFoundError(f"Missing '{filename}' in {MODELS_DIR}. Did you run the training notebook?")
    return path

base_models_dict = joblib.load(_require_model_file('ensemble_base_models_neural.joblib'))
meta_model = keras.models.load_model(_require_model_file('meta_learner_neural.h5'), compile=False)
neural_network_model = keras.models.load_model(_require_model_file('residual_neural.h5'), compile=False)

xgb_deep = base_models_dict['xgb_deep']
xgb_shallow = base_models_dict['xgb_shallow']
lgbm_fast = base_models_dict['lgbm_fast']
catboost_robust = base_models_dict['catboost_robust']

print("Ensemble components loaded successfully")


Ensemble components loaded successfully


In [4]:
# ensemble_predict(): Combines base model predictions via meta-learner stacking
# Parameters: X (DataFrame or array) - input features
# Returns: array - probability predictions from stacked ensemble
def ensemble_predict(X):
    if isinstance(X, pd.DataFrame):
        X = X.values
    
    pred_xgb_deep = xgb_deep.predict_proba(X)[:, 1]
    pred_xgb_shallow = xgb_shallow.predict_proba(X)[:, 1]
    pred_lgbm = lgbm_fast.predict_proba(X)[:, 1]
    pred_catboost = catboost_robust.predict_proba(X)[:, 1]
    pred_neural = neural_network_model.predict(X, verbose=0).ravel()
    
    base_preds = np.column_stack([pred_xgb_deep, pred_xgb_shallow, pred_lgbm, pred_catboost, pred_neural])
    final_pred = meta_model.predict(base_preds, verbose=0).ravel()
    
    return final_pred


## Model-Specific SHAP Comparison


### Background Data Preparation


In [5]:
background_data = X_train.sample(n=min(200, len(X_train)), random_state=42).values
test_sample_size = min(100, len(X_test))
X_test_sample = X_test.iloc[:test_sample_size].values

print(f"Background data size: {background_data.shape[0]}")
print(f"Test sample size: {test_sample_size}")


Background data size: 200
Test sample size: 100


### Individual Model SHAP Computation


In [6]:
# Compute SHAP values separately for each base model
# This lets us see which models rely on which features differently
# PermutationExplainer works by shuffling features and measuring impact
model_explainers = {}
model_shap_values = {}

print("Computing SHAP values for individual models...")

# XGBoost Deep
explainer_xgb_deep = shap.PermutationExplainer(
    lambda X: xgb_deep.predict_proba(X)[:, 1],
    background_data
)
shap_xgb_deep = explainer_xgb_deep(X_test_sample)
model_shap_values['xgb_deep'] = shap_xgb_deep.values
print("  XGBoost Deep: Done")

# XGBoost Shallow
explainer_xgb_shallow = shap.PermutationExplainer(
    lambda X: xgb_shallow.predict_proba(X)[:, 1],
    background_data
)
shap_xgb_shallow = explainer_xgb_shallow(X_test_sample)
model_shap_values['xgb_shallow'] = shap_xgb_shallow.values
print("  XGBoost Shallow: Done")

# LightGBM
explainer_lgbm = shap.PermutationExplainer(
    lambda X: lgbm_fast.predict_proba(X)[:, 1],
    background_data
)
shap_lgbm = explainer_lgbm(X_test_sample)
model_shap_values['lgbm'] = shap_lgbm.values
print("  LightGBM: Done")

# CatBoost
explainer_catboost = shap.PermutationExplainer(
    lambda X: catboost_robust.predict_proba(X)[:, 1],
    background_data
)
shap_catboost = explainer_catboost(X_test_sample)
model_shap_values['catboost'] = shap_catboost.values
print("  CatBoost: Done")

# Neural Network
explainer_nn = shap.PermutationExplainer(
    lambda X: neural_network_model.predict(X, verbose=0).ravel(),
    background_data
)
shap_nn = explainer_nn(X_test_sample)
model_shap_values['neural_network'] = shap_nn.values
print("  Neural Network: Done")

print("All SHAP values computed")


Computing SHAP values for individual models...


PermutationExplainer explainer: 101it [00:11,  2.95it/s]                         


  XGBoost Deep: Done
  XGBoost Shallow: Done


Exception in thread Thread-5 (_readerthread):
Traceback (most recent call last):
  File "d:\Conda\envs\final_last\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "d:\Conda\envs\final_last\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "d:\Conda\envs\final_last\lib\subprocess.py", line 1515, in _readerthread
    buffer.append(fh.read())
  File "d:\Conda\envs\final_last\lib\codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xce in position 4: invalid continuation byte


  LightGBM: Done


PermutationExplainer explainer: 101it [00:19,  2.57it/s]                         


  CatBoost: Done


PermutationExplainer explainer: 101it [06:40,  4.09s/it]                         

  Neural Network: Done
All SHAP values computed


### Feature Importance Comparison Across Models


In [7]:
# Calculate average feature importance for each model
# Then compare across models to see which features are consistently important
model_importances = {}

for model_name, shap_vals in model_shap_values.items():
    # Average absolute SHAP values = feature importance for this model
    importance = np.mean(np.abs(shap_vals), axis=0)
    model_importances[model_name] = importance

importance_df = pd.DataFrame(model_importances, index=feature_names)
# Sort by average importance across all models
feature_mean_importance = importance_df.mean(axis=1)
importance_df = importance_df.loc[feature_mean_importance.sort_values(ascending=False).index]

print("Top 10 features by average importance across all models:")
print(importance_df.head(10))


Top 10 features by average importance across all models:
                             xgb_deep  xgb_shallow      lgbm  catboost  \
loan_grade                   0.086685     0.104630  0.095004  0.099147   
loan_percent_income          0.078916     0.084808  0.086118  0.088837   
person_income                0.087958     0.073468  0.081706  0.063153   
loan_intent_VENTURE          0.051779     0.050734  0.053481  0.047652   
person_home_ownership_RENT   0.045805     0.039029  0.041822  0.040184   
person_home_ownership_OWN    0.022990     0.027919  0.025549  0.024274   
loan_int_rate                0.025680     0.022979  0.025413  0.024311   
loan_intent_HOMEIMPROVEMENT  0.020085     0.014059  0.018264  0.024699   
loan_amnt                    0.025162     0.008627  0.011816  0.007262   
loan_intent_EDUCATION        0.016257     0.016403  0.014982  0.015001   

                             neural_network  
loan_grade                         0.078515  
loan_percent_income                0

### Model Agreement Visualization


In [8]:
top_n_features = 12
top_features = importance_df.head(top_n_features).index.tolist()

fig, ax = plt.subplots(figsize=(14, 8))
x = np.arange(len(top_features))
width = 0.15

models_list = ['xgb_deep', 'xgb_shallow', 'lgbm', 'catboost', 'neural_network']
colors = ['#E63946', '#F77F00', '#FCBF49', '#06A77D', '#2E86AB']

for idx, model_name in enumerate(models_list):
    values = [importance_df.loc[feat, model_name] for feat in top_features]
    ax.bar(x + idx * width, values, width, label=model_name.replace('_', ' ').title(), 
           alpha=0.85, color=colors[idx])

ax.set_xlabel('Features', fontsize=13, fontweight='bold')
ax.set_ylabel('Mean |SHAP Value|', fontsize=13, fontweight='bold')
ax.set_title('Feature Importance Comparison Across Base Models', fontsize=15, fontweight='bold')
ax.set_xticks(x + width * 2)
ax.set_xticklabels(top_features, rotation=45, ha='right', fontsize=10)
ax.legend(fontsize=11, loc='upper right')
ax.grid(axis='y', alpha=0.3, linestyle='--')

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'model_shap_comparison.png'), dpi=300, bbox_inches='tight')
plt.close()


### Model Disagreement Analysis


In [9]:
# Measure model disagreement: which features do models disagree about?
# High coefficient of variation = models have very different opinions on this feature
# Low CV = models agree this feature is important (or unimportant)
importance_std = importance_df.std(axis=1)
importance_mean = importance_df.mean(axis=1)

# Coefficient of variation = std / mean (normalized disagreement measure)
disagreement_df = pd.DataFrame({
    'feature': feature_names,
    'mean_importance': importance_mean.values,
    'std_importance': importance_std.values,
    'coefficient_of_variation': (importance_std / (importance_mean + 1e-9)).values
}).sort_values('coefficient_of_variation', ascending=False)

print("Top 10 features with highest model disagreement (CV):")
print(disagreement_df.head(10)[['feature', 'mean_importance', 'coefficient_of_variation']])


Top 10 features with highest model disagreement (CV):
                        feature  mean_importance  coefficient_of_variation
15          loan_intent_VENTURE         0.001826                  0.832700
16    cb_person_default_on_file         0.000275                  0.694993
8   person_home_ownership_OTHER         0.017157                  0.659193
14         loan_intent_PERSONAL         0.003013                  0.556767
11        loan_intent_EDUCATION         0.011671                  0.318773
12  loan_intent_HOMEIMPROVEMENT         0.011455                  0.309435
2             person_emp_length         0.068688                  0.290131
7                    loan_grade         0.018295                  0.240720
6    cb_person_cred_hist_length         0.022392                  0.225160
5           loan_percent_income         0.023097                  0.216745


### Disagreement Heatmap


In [10]:
top_disagree_features = disagreement_df.head(10)['feature'].tolist()
disagreement_subset = importance_df.loc[top_disagree_features]

plt.figure(figsize=(10, 8))
sns.heatmap(disagreement_subset, annot=True, fmt='.4f', cmap='YlOrRd', 
            xticklabels=[m.replace('_', ' ').title() for m in models_list],
            yticklabels=top_disagree_features, cbar_kws={'label': 'Feature Importance'})
plt.title('Model Disagreement: Top 10 Features with Highest Variance', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'model_disagreement_heatmap.png'), dpi=300, bbox_inches='tight')
plt.close()


## Sensitivity Analysis


### Feature Perturbation Setup


In [11]:
sensitivity_sample_size = min(50, len(X_test))
X_sensitivity = X_test.iloc[:sensitivity_sample_size].values
original_predictions = ensemble_predict(X_sensitivity)

print(f"Sensitivity analysis sample size: {sensitivity_sample_size}")


Sensitivity analysis sample size: 50


### Sensitivity Computation


In [12]:
# Sensitivity analysis: test how much predictions change when we tweak each feature
# More sensitive features = small changes cause big prediction changes
# Less sensitive = predictions stay stable when feature changes
perturbation_levels = [0.1, 0.2]  # Try ±10% and ±20% changes
feature_sensitivity = np.zeros(len(feature_names))

for feat_idx in range(len(feature_names)):
    sensitivity_scores = []
    
    for pert_level in perturbation_levels:
        X_perturbed_plus = X_sensitivity.copy()
        X_perturbed_minus = X_sensitivity.copy()
        
        # Increase and decrease feature by same percentage
        X_perturbed_plus[:, feat_idx] *= (1 + pert_level)
        X_perturbed_minus[:, feat_idx] *= (1 - pert_level)
        
        # Measure prediction change
        pred_plus = ensemble_predict(X_perturbed_plus)
        pred_minus = ensemble_predict(X_perturbed_minus)
        
        diff_plus = np.mean(np.abs(original_predictions - pred_plus))
        diff_minus = np.mean(np.abs(original_predictions - pred_minus))
        
        # Average the two directions
        sensitivity_scores.append((diff_plus + diff_minus) / 2)
    
    # Average across perturbation levels
    feature_sensitivity[feat_idx] = np.mean(sensitivity_scores)

sensitivity_df = pd.DataFrame({
    'feature': feature_names,
    'sensitivity': feature_sensitivity
}).sort_values('sensitivity', ascending=False)

print("Top 10 most sensitive features:")
print(sensitivity_df.head(10))


Top 10 most sensitive features:
                        feature  sensitivity
1                 person_income     0.035567
7                    loan_grade     0.025388
5           loan_percent_income     0.019726
10   person_home_ownership_RENT     0.009533
2             person_emp_length     0.007578
4                 loan_int_rate     0.007157
3                     loan_amnt     0.007076
15          loan_intent_VENTURE     0.003998
12  loan_intent_HOMEIMPROVEMENT     0.003363
11        loan_intent_EDUCATION     0.002718


### Sensitivity Visualization


In [13]:
top_sensitive_features = sensitivity_df.head(10)

fig, ax = plt.subplots(figsize=(12, 7))
ax.barh(range(len(top_sensitive_features)), top_sensitive_features['sensitivity'], 
        color='#E63946', alpha=0.8)
ax.set_yticks(range(len(top_sensitive_features)))
ax.set_yticklabels(top_sensitive_features['feature'], fontsize=11)
ax.set_xlabel('Sensitivity Score (Mean Prediction Change)', fontsize=13, fontweight='bold')
ax.set_title('Top 10 Most Sensitive Features', fontsize=15, fontweight='bold')
ax.grid(axis='x', alpha=0.3, linestyle='--')
ax.invert_yaxis()

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'feature_sensitivity_ranking.png'), dpi=300, bbox_inches='tight')
plt.close()


### Sensitivity Curves for Top Features


In [14]:
# Create sensitivity curves: show how predictions change as feature varies
# X-axis = feature change percentage, Y-axis = average prediction
top_3_sensitive = sensitivity_df.head(3)['feature'].tolist()

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, feat_name in enumerate(top_3_sensitive):
    feat_idx = feature_names.index(feat_name)
    
    # Vary feature from -30% to +30% around median value
    base_value = np.median(X_sensitivity[:, feat_idx])
    pert_range = np.linspace(-0.3, 0.3, 20)
    
    sensitivity_curve = []
    for pert in pert_range:
        # Set all samples to same feature value (base * (1 + pert))
        X_pert = X_sensitivity.copy()
        X_pert[:, feat_idx] = base_value * (1 + pert)
        pred_pert = ensemble_predict(X_pert)
        sensitivity_curve.append(np.mean(pred_pert))
    
    axes[idx].plot(pert_range * 100, sensitivity_curve, linewidth=2.5, color='#2E86AB', marker='o', markersize=4)
    axes[idx].axhline(y=np.mean(original_predictions), color='r', linestyle='--', alpha=0.7, label='Original Mean')
    axes[idx].set_xlabel(f'{feat_name} Perturbation (%)', fontsize=11, fontweight='bold')
    axes[idx].set_ylabel('Mean Prediction', fontsize=11)
    axes[idx].set_title(f'Sensitivity Curve: {feat_name}', fontsize=12, fontweight='bold')
    axes[idx].grid(True, alpha=0.3)
    axes[idx].legend(fontsize=9)

plt.suptitle('Sensitivity Curves for Top 3 Most Sensitive Features', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'sensitivity_curves.png'), dpi=300, bbox_inches='tight')
plt.close()


## Decision Boundary Analysis


### Dimensionality Reduction Setup


In [15]:
db_sample_size = min(500, len(X_test))
X_db_sample = X_test.iloc[:db_sample_size].values
y_db_sample = y_test.iloc[:db_sample_size].values if hasattr(y_test, 'iloc') else y_test[:db_sample_size]
predictions_db = ensemble_predict(X_db_sample)

print(f"Decision boundary analysis sample size: {db_sample_size}")


Decision boundary analysis sample size: 500


### PCA Projection


In [16]:
# PCA: reduces 17 features to 2 dimensions for visualization
# Finds the 2 directions that capture most variance in the data
# This lets us see decision boundaries in 2D plots
scaler = StandardScaler()
X_db_scaled = scaler.fit_transform(X_db_sample)

pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_db_scaled)

print(f"PCA explained variance ratio: {pca.explained_variance_ratio_}")
print(f"Total variance explained: {pca.explained_variance_ratio_.sum():.3f}")


PCA explained variance ratio: [0.14061448 0.1327594 ]
Total variance explained: 0.273


### Decision Boundary Visualization (PCA)


In [17]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Color by actual label
scatter1 = axes[0].scatter(X_pca[:, 0], X_pca[:, 1], c=y_db_sample, 
                           cmap='RdYlGn', alpha=0.6, s=30, edgecolors='black', linewidth=0.5)
axes[0].set_xlabel('First Principal Component', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Second Principal Component', fontsize=12, fontweight='bold')
axes[0].set_title('PCA Projection: Actual Labels', fontsize=13, fontweight='bold')
plt.colorbar(scatter1, ax=axes[0], label='Actual Risk (0=Low, 1=High)')
axes[0].grid(True, alpha=0.3)

# Plot 2: Color by prediction probability
scatter2 = axes[1].scatter(X_pca[:, 0], X_pca[:, 1], c=predictions_db, 
                          cmap='RdYlGn', alpha=0.6, s=30, edgecolors='black', linewidth=0.5)
axes[1].set_xlabel('First Principal Component', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Second Principal Component', fontsize=12, fontweight='bold')
axes[1].set_title('PCA Projection: Predicted Probabilities', fontsize=13, fontweight='bold')
plt.colorbar(scatter2, ax=axes[1], label='Predicted Risk Probability')
axes[1].grid(True, alpha=0.3)

plt.suptitle('Decision Boundary Analysis using PCA', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'decision_boundary_pca.png'), dpi=300, bbox_inches='tight')
plt.close()


### Risk Threshold Regions


In [18]:
# Group predictions into risk categories for visualization
# Low risk: <30%, Medium: 30-70%, High: ≥70%
risk_categories = np.zeros(len(predictions_db))
risk_categories[predictions_db < 0.3] = 0  # Low risk
risk_categories[(predictions_db >= 0.3) & (predictions_db < 0.7)] = 1  # Medium risk
risk_categories[predictions_db >= 0.7] = 2  # High risk

fig, ax = plt.subplots(figsize=(10, 8))
scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], c=risk_categories, 
                    cmap='RdYlGn', alpha=0.7, s=40, edgecolors='black', linewidth=0.5)
ax.set_xlabel('First Principal Component', fontsize=13, fontweight='bold')
ax.set_ylabel('Second Principal Component', fontsize=13, fontweight='bold')
ax.set_title('Risk Threshold Regions (Low/Medium/High)', fontsize=14, fontweight='bold')
cbar = plt.colorbar(scatter, ax=ax, ticks=[0, 1, 2])
cbar.set_ticklabels(['Low Risk (<0.3)', 'Medium Risk (0.3-0.7)', 'High Risk (≥0.7)'])
cbar.set_label('Risk Category', fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'risk_threshold_regions.png'), dpi=300, bbox_inches='tight')
plt.close()


## Explanation Consistency Analysis


### Instance Clustering


In [19]:
consistency_sample_size = min(200, len(X_test))
X_consistency = X_test.iloc[:consistency_sample_size].values

# Cluster similar instances together to test explanation consistency
# If instances are similar, their SHAP explanations should also be similar
scaler_consistency = StandardScaler()
X_consistency_scaled = scaler_consistency.fit_transform(X_consistency)

# Group into 5 clusters based on feature similarity
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X_consistency_scaled)

print(f"Clustered {consistency_sample_size} instances into {n_clusters} clusters")
print(f"Cluster sizes: {np.bincount(cluster_labels)}")


Clustered 200 instances into 5 clusters
Cluster sizes: [26 36 39 32 67]


### SHAP Values for Consistency Analysis


In [20]:
# Compute SHAP values for clustered instances
# We'll check if instances in same cluster have similar SHAP values (consistent explanations)
explainer_ensemble = shap.PermutationExplainer(ensemble_predict, background_data)
shap_consistency = explainer_ensemble(X_consistency)
shap_consistency_values = shap_consistency.values

print(f"SHAP values computed for {consistency_sample_size} instances")


PermutationExplainer explainer: 201it [19:24,  5.85s/it]                         

SHAP values computed for 200 instances


### Consistency Metrics Computation


In [21]:
# Check explanation consistency: do similar instances get similar explanations?
# For each cluster, measure how much SHAP values vary within the cluster
# Low variation = consistent explanations (good), high variation = inconsistent (bad)
cluster_consistency = {}

for cluster_id in range(n_clusters):
    cluster_mask = cluster_labels == cluster_id
    cluster_shap = shap_consistency_values[cluster_mask]
    
    if len(cluster_shap) > 1:
        # Average SHAP importance for features in this cluster
        mean_shap = np.mean(np.abs(cluster_shap), axis=0)
        # How much do SHAP values vary within this cluster?
        std_shap = np.std(np.abs(cluster_shap), axis=0)
        # Coefficient of variation: normalized measure of consistency
        # Lower CV = more consistent (less variation relative to mean)
        cv_shap = std_shap / (mean_shap + 1e-9)
        
        cluster_consistency[cluster_id] = {
            'size': int(np.sum(cluster_mask)),
            'mean_importance': mean_shap.tolist(),
            'std_importance': std_shap.tolist(),
            'cv': cv_shap.tolist(),
            'mean_cv': float(np.mean(cv_shap))
        }

print("Cluster consistency metrics:")
for cluster_id, metrics in cluster_consistency.items():
    print(f"  Cluster {cluster_id}: size={metrics['size']}, mean_CV={metrics['mean_cv']:.4f}")


Cluster consistency metrics:
  Cluster 0: size=26, mean_CV=0.8686
  Cluster 1: size=36, mean_CV=1.2416
  Cluster 2: size=39, mean_CV=1.5320
  Cluster 3: size=32, mean_CV=1.0487
  Cluster 4: size=67, mean_CV=1.0565


### Consistency Visualization


In [22]:
top_features_consistency = importance_df.head(8).index.tolist()
top_feature_indices = [feature_names.index(f) for f in top_features_consistency]

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for cluster_id in range(min(4, n_clusters)):
    if cluster_id not in cluster_consistency:
        continue
    
    metrics = cluster_consistency[cluster_id]
    mean_imp = np.array(metrics['mean_importance'])
    std_imp = np.array(metrics['std_importance'])
    
    top_mean = mean_imp[top_feature_indices]
    top_std = std_imp[top_feature_indices]
    
    x_pos = np.arange(len(top_features_consistency))
    axes[cluster_id].bar(x_pos, top_mean, yerr=top_std, alpha=0.7, 
                        color='#2E86AB', capsize=5)
    axes[cluster_id].set_xlabel('Features', fontsize=11, fontweight='bold')
    axes[cluster_id].set_ylabel('Mean |SHAP| ± Std', fontsize=11)
    axes[cluster_id].set_title(f'Cluster {cluster_id} (n={metrics["size"]}, CV={metrics["mean_cv"]:.3f})', 
                              fontsize=12, fontweight='bold')
    axes[cluster_id].set_xticks(x_pos)
    axes[cluster_id].set_xticklabels(top_features_consistency, rotation=45, ha='right', fontsize=9)
    axes[cluster_id].grid(axis='y', alpha=0.3)

plt.suptitle('Explanation Consistency Across Clusters', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'explanation_consistency.png'), dpi=300, bbox_inches='tight')
plt.close()


### Overall Consistency Summary


In [23]:
overall_mean_cv = np.mean([m['mean_cv'] for m in cluster_consistency.values()])
consistency_scores = [m['mean_cv'] for m in cluster_consistency.values()]

print(f"Overall mean coefficient of variation: {overall_mean_cv:.4f}")
print(f"Consistency range: [{min(consistency_scores):.4f}, {max(consistency_scores):.4f}]")

# Lower CV = more consistent
if overall_mean_cv < 0.5:
    consistency_level = "High"
elif overall_mean_cv < 1.0:
    consistency_level = "Medium"
else:
    consistency_level = "Low"

print(f"Consistency level: {consistency_level}")


Overall mean coefficient of variation: 1.1495
Consistency range: [0.8686, 1.5320]
Consistency level: Low


## Results Export and Summary


### Save Model-Specific SHAP Values


In [24]:
model_shap_summary = {}

for model_name, shap_vals in model_shap_values.items():
    importance = np.mean(np.abs(shap_vals), axis=0)
    top_features = pd.DataFrame({
        'feature': feature_names,
        'importance': importance
    }).sort_values('importance', ascending=False).head(10)
    
    model_shap_summary[model_name] = top_features.to_dict('records')

with open(os.path.join(OUTPUT_DIR, 'model_specific_shap_summary.json'), 'w') as f:
    json.dump(model_shap_summary, f, indent=2)


### Generate Comprehensive Report


In [25]:
advanced_report = {
    'timestamp': pd.Timestamp.now().isoformat(),
    'sample_sizes': {
        'model_shap_comparison': int(test_sample_size),
        'sensitivity_analysis': int(sensitivity_sample_size),
        'decision_boundary': int(db_sample_size),
        'consistency_analysis': int(consistency_sample_size)
    },
    'model_specific_analysis': {
        'top_features_by_model': model_shap_summary,
        'model_disagreement': {
            'top_disagree_features': disagreement_df.head(10)[['feature', 'coefficient_of_variation']].to_dict('records')
        }
    },
    'sensitivity_analysis': {
        'top_sensitive_features': sensitivity_df.head(10).to_dict('records'),
        'max_sensitivity': float(sensitivity_df.iloc[0]['sensitivity']),
        'mean_sensitivity': float(sensitivity_df['sensitivity'].mean())
    },
    'decision_boundary_analysis': {
        'pca_variance_explained': {
            'pc1': float(pca.explained_variance_ratio_[0]),
            'pc2': float(pca.explained_variance_ratio_[1]),
            'total': float(pca.explained_variance_ratio_.sum())
        },
        'risk_distribution': {
            'low_risk': int(np.sum(risk_categories == 0)),
            'medium_risk': int(np.sum(risk_categories == 1)),
            'high_risk': int(np.sum(risk_categories == 2))
        }
    },
    'explanation_consistency': {
        'n_clusters': int(n_clusters),
        'overall_mean_cv': float(overall_mean_cv),
        'consistency_level': consistency_level,
        'cluster_details': cluster_consistency
    }
}

with open(os.path.join(OUTPUT_DIR, 'advanced_explainability_report.json'), 'w') as f:
    json.dump(advanced_report, f, indent=2)

print("Advanced Explainability Report Summary:")
print(f"  Model SHAP Comparison: {test_sample_size} samples")
print(f"  Sensitivity Analysis: {sensitivity_sample_size} samples")
print(f"  Decision Boundary: {db_sample_size} samples, {pca.explained_variance_ratio_.sum():.1%} variance explained")
print(f"  Consistency Analysis: {consistency_sample_size} samples, {consistency_level} consistency")
print(f"  Top Sensitive Feature: {sensitivity_df.iloc[0]['feature']}")
print(f"  Most Disagreed Feature: {disagreement_df.iloc[0]['feature']}")
print("\nAll results saved to artifacts/04b_images/")


Advanced Explainability Report Summary:
  Model SHAP Comparison: 100 samples
  Sensitivity Analysis: 50 samples
  Decision Boundary: 500 samples, 27.3% variance explained
  Consistency Analysis: 200 samples, Low consistency
  Top Sensitive Feature: person_income
  Most Disagreed Feature: loan_intent_VENTURE

All results saved to artifacts/04b_images/
